gathering data from the tables to be used in analysis - not all columns in the tables are needed for our purposes.  
SQL tables being used:  
    pums_2017
    pums_2018 
    puma_names_2010
    wa_jobs_2017
    wa_geo_xwalk 
    ct_puma_xwalk

In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
DBNAME = "opportunity_youth"
conn = psycopg2.connect(dbname=DBNAME)

In [ ]:
WAresp = pd.read_sql("""
                   SELECT agep
                   FROM pums_2017
                   WHERE st = '53'
                   """,conn)

In [ ]:
WAresp.plot.hist(bins = 20);

In [ ]:
WAresp.agep.mean()

In [ ]:
WAresp.agep.mode()

In [ ]:
KingCounty = pd.read_sql("""
                         SELECT agep
                         FROM pums_2017 p
                         JOIN puma_names_2010 n 
                              ON p.puma = n.puma
                         WHERE state_name = 'Washington'
                         AND puma_name like ('%King%County%')""",conn)
KingCounty

In [ ]:
KingCounty.plot.hist(bins = 20);

In [ ]:
KingCounty.agep.median()

In [ ]:
KingCounty.agep.mode()

In [ ]:
SKC = pd.read_sql("""
                  SELECT agep
                  FROM pums_2017 p
                  JOIN puma_names_2010 n on p.puma = n.puma
                  WHERE p.puma in ('11610','11611','11612','11613','11614','11615')""",conn)
SKC

In [ ]:
SKC.agep.median()

In [ ]:
SKC.agep.mode()

In [ ]:
SKC.plot.hist(bins=20);

In [ ]:
pd.read_sql("""
            SELECT agep, puma
            FROM pums_2018
            LIMIT 10""",conn)

In [ ]:
dt2017 = pd.read_sql("""
            SELECT p.puma, rtrim(puma_name), agep, 
            CASE WHEN p.sex = '1' THEN 'male'
                 ELSE 'female'
                 END as gender,
            CASE WHEN p.racasn = '1' THEN 'asian'
                 WHEN p.racblk = '1' THEN 'black or African American'
                 WHEN p.racwht = '1' THEN 'white'
                 WHEN p.hisp <> '1' THEN 'hispanic' 
                 END as race_or_ethnicity,
            CASE WHEN p.sch = '1' THEN 'Has not attended in last 3 months'
                 WHEN p.sch IN ('2','3') THEN 'public or private school or college and homeschool'
                 END as School,
            CASE WHEN p.schl in ('16','17') THEN 'HS diploma or GED'
                 WHEN p.schl in ('18','19','20','21','22','23','24') THEN 'post HS education'
                 ELSE 'No HS or GED'
                 END as Education_attainment,
            CASE WHEN p.esr in ('1','2') THEN 'employed'
                 WHEN p.esr in ('3','6') THEN 'Unemployed/not in labor force'
                 WHEN p.esr IN ('4','5') THEN 'Armed Forces'
                 END as Employment_status,
            CASE WHEN p.dis = '1' THEN 'reported disability'
                 ELSE 'no reported disability'
                 END as disability_status,
            CASE WHEN p.ddrs = '1' THEN 'Self-care difficulty'
                 WHEN p.dear = '1' THEN 'Hearing difficulty'
                 WHEN p.deye = '1' THEN 'Vision difficulty'
                 WHEN p.dphy = '1' THEN 'Ambulatory difficulty'
                 WHEN p.drem = '1' THEN 'Cognitive difficulty'
                 ELSE 'None reported' 
                 END as disability_type,
            CASE WHEN p.cit IN ('1','2','3','4') THEN 'US Citizen'
                 WHEN p.cit = '5' THEN 'Not US Citizen' 
                 END as Citizenship,
            CASE WHEN p.eng IN ('1','2') THEN 'English Speaker'
                 ELSE 'Poor or No English language'
                 END as English_language,
            CASE WHEN p.wkl = '1' THEN 'employed in the last year'
                 WHEN p.wkl in ('2','3') THEN 'not employed in the last year' 
                 END AS last_employed,
            CASE WHEN p.esp in ('1','2','3','5','6','7','8') THEN 'One or both parents in labor force'
                 WHEN p.esp = '4' THEN 'Neither parent in labor force'
                 END as Parent_employment,
            CASE WHEN p.mar = '1' THEN 'married'
                 WHEN p.mar = '5' THEN 'never married'
                 ELSE 'divorced, widowed, or separated' 
                 END as marital_status,
            CASE WHEN p.fer = '1' THEN 'yes' 
                 END AS Child_born_las_year,
            CASE WHEN p.paoc in ('1','2','3') THEN 'own child(ren)' 
                 ELSE 'no own children' 
                 END as Presence_of_children, 
            CASE WHEN p.mig = '1' THEN 'lived here last year' 
                 ELSE 'moved here last year' 
                 END AS Moved,
            CASE WHEN p.hicov = '1' THEN 'yes' 
                 ELSE 'no' 
                 END as Has_Health_Insurance,
            pap as Public_Assistance_Income_past_year,        
            pwgtp as Weight
            FROM pums_2017 p
            JOIN puma_names_2010 n on p.puma = n.puma
            WHERE state_name = 'Washington'
            AND p.puma IN ('11610','11611', '11612','11613','11614','11615')
            AND p.sch = '1' 
            AND p.esr IN ('3','6')
            AND agep BETWEEN 16 AND 24
            GROUP BY p.puma,n.puma_name,agep,p.sex,p.racasn,p.racblk,p.racwht,p.hisp,p.sch,p.schl,
                     p.esr,p.dis,p.ddrs,p.dear,p.deye,p.dphy,p.drem,p.cit,p.eng,p.wkl,p.esp,
                     p.mar,p.fer,p.paoc,p.mig,p.hicov,p.pap,p.pwgtp
            ORDER BY school, agep DESC""", conn)
dt2017

In [72]:
dt2018 = pd.read_sql("""
            SELECT p.puma, rtrim(puma_name), agep, 
                        CASE WHEN p.sex = '1' THEN 'male'
                 ELSE 'female'
                 END as gender,
            CASE WHEN p.racasn = '1' THEN 'asian'
                 WHEN p.racblk = '1' THEN 'black or African American'
                 WHEN p.racwht = '1' THEN 'white'
                 WHEN p.hisp <> '1' THEN 'hispanic' 
                 END as race_or_ethnicity,
            CASE WHEN p.sch = '1' THEN 'Has not attended in last 3 months'
                 WHEN p.sch IN ('2','3') THEN 'public or private school or college and homeschool'
                 END as School,
            CASE WHEN p.schl in ('16','17') THEN 'HS diploma or GED'
                 WHEN p.schl in ('18','19','20','21','22','23','24') THEN 'post HS education'
                 ELSE 'No HS or GED'
                 END as Education_attainment,
            CASE WHEN p.esr in ('1','2') THEN 'employed'
                 WHEN p.esr in ('3','6') THEN 'Unemployed/not in labor force'
                 WHEN p.esr IN ('4','5') THEN 'Armed Forces'
                 END as Employment_status,
            CASE WHEN p.dis = '1' THEN 'reported disability'
                 ELSE 'no reported disability'
                 END as disability_status,
            CASE WHEN p.ddrs = '1' THEN 'Self-care difficulty'
                 WHEN p.dear = '1' THEN 'Hearing difficulty'
                 WHEN p.deye = '1' THEN 'Vision difficulty'
                 WHEN p.dphy = '1' THEN 'Ambulatory difficulty'
                 WHEN p.drem = '1' THEN 'Cognitive difficulty'
                 ELSE 'None reported' 
                 END as disability_type,
            CASE WHEN p.cit IN ('1','2','3','4') THEN 'US Citizen'
                 WHEN p.cit = '5' THEN 'Not US Citizen' 
                 END as Citizenship,
            CASE WHEN p.eng IN ('3','4') THEN 'Poor or No English Skills'
                 ELSE 'English Speaker'
                 END as English_language,
            CASE WHEN p.wkl = '1' THEN 'employed in the last year'
                 WHEN p.wkl in ('2','3') THEN 'not employed in the last year' 
                 END AS last_employed,
            CASE WHEN p.esp in ('1','2','3','5','6','7','8') THEN 'One or both parents in labor force'
                 WHEN p.esp = '4' THEN 'Neither parent in labor force'
                 END as Parent_employment,
            CASE WHEN p.mar = '1' THEN 'married'
                 WHEN p.mar = '5' THEN 'never married'
                 ELSE 'divorced, widowed, or separated' 
                 END as marital_status,
            CASE WHEN p.fer = '1' THEN 'yes' 
                 END AS Child_born_las_year,
            CASE WHEN p.paoc in ('1','2','3') THEN 'own child(ren)' 
                 ELSE 'no own children' 
                 END as Presence_of_children, 
            CASE WHEN p.mig = '1' THEN 'lived here last year' 
                 ELSE 'moved here last year' 
                 END AS Moved,
            CASE WHEN p.hicov = '1' THEN 'yes' 
                 ELSE 'no' 
                 END as Has_Health_Insurance,
            pap as Public_Assistance_Income_past_year,        
            pwgtp as Weight
            FROM pums_2018 p
            JOIN puma_names_2010 n on p.puma = n.puma
            WHERE state_name = 'Washington'
            AND p.puma IN ('11610','11611', '11612','11613','11614','11615')
            AND p.sch = '1' 
            AND p.esr IN ('3','6')
            AND agep BETWEEN 16 AND 24
            GROUP BY p.puma,n.puma_name,agep,p.sex,p.racasn,p.racblk,p.racwht,p.hisp,p.sch,p.schl,
                     p.esr,p.dis,p.ddrs,p.dear,p.deye,p.dphy,p.drem,p.cit,p.eng,p.wkl,p.esp,
                     p.mar,p.fer,p.paoc,p.mig,p.hicov,p.pap,p.pwgtp
            ORDER BY school, agep DESC""", conn)
dt2018

,puma,rtrim,agep,gender,race_or_ethnicity,school,education_attainment,employment_status,disability_status,disability_type,...,english_language,last_employed,parent_employment,marital_status,child_born_las_year,presence_of_children,moved,has_health_insurance,public_assistance_income_past_year,weight
0,11614,King County (Southwest)--Auburn City & Lakeland,24.0,female,asian,Has not attended in last 3 months,No HS or GED,Unemployed/not in labor force,no reported disability,None reported,...,English Speaker,not employed in the last year,None,never married,None,no own children,lived here last year,yes,0.0,21.0
1,11614,King County (Southwest)--Auburn City & Lakeland,24.0,female,black or African American,Has not attended in last 3 months,post HS education,Unemployed/not in labor force,no reported disability,None reported,...,English Speaker,employed in the last year,None,married,yes,no own children,moved here last year,yes,0.0,8.0
2,11614,King County (Southwest)--Auburn City & Lakeland,24.0,female,white,Has not attended in last 3 months,HS diploma or GED,Unemployed/not in labor force,no reported disability,None reported,...,English Speaker,not employed in the last year,None,never married,None,no own children,lived here last year,no,0.0,3.0
3,11614,King County (Southwest)--Auburn City & Lakeland,24.0,female,white,Has not attended in last 3 months,post HS education,Unemployed/not in labor force,no reported disability,None reported,...,English Speaker,employed in the last year,None,never married,None,no own children,moved here last year,yes,0.0,23.0
4,11614,King County (Southwest)--Auburn City & Lakeland,24.0,male,hispanic,Has not attended in last 3 months,No HS or GED,Unemployed/not in labor force,no reported disability,None reported,...,English Speaker,not employed in the last year,None,never married,None,no own children,lived here last year,no,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,11612,"King County (Far Southwest)--Federal Way, Des ...",16.0,female,white,Has not attended in last 3 months,No HS or GED,Unemployed/not in labor force,no reported disability,None reported,...,English Speaker,not employed in the last year,One or both parents in labor force,never married,None,no own children,lived here last year,yes,0.0,21.0
362,11614,King County (Southwest)--Auburn City & Lakeland,16.0,male,white,Has not attended in last 3 months,HS diploma or GED,Unemployed/not in labor force,no reported disability,None reported,...,English Speaker,not employed in the last year,One or both parents in labor force,never married,None,no own children,lived here last year,yes,0.0,12.0
363,11614,King County (Southwest)--Auburn City & Lakeland,16.0,male,white,Has not attended in last 3 months,No HS or GED,Unemployed/not in labor force,no reported disability,None reported,...,English Speaker,not employed in the last year,One or both parents in labor force,never married,None,no own children,lived here last year,yes,0.0,24.0
364,11615,"King County (Southeast)--Maple Valley, Covingt...",16.0,female,white,Has not attended in last 3 months,No HS or GED,Unemployed/not in labor force,reported disability,None reported,...,English Speaker,not employed in the last year,One or both parents in labor force,never married,None,no own children,lived here last year,yes,0.0,13.0


In [70]:
dt2017.groupby('english_language')['weight'].agg(sum)

english_language
English Speaker                3467.0
Poor or No English language    7147.0
Name: weight, dtype: float64

In [ ]:
dt2018.groupby('gender')['weight'].agg(sum)

In [ ]:
dt2017.groupby('race_or_ethnicity')['weight'].agg(sum)

In [ ]:
dt2018.columns

In [ ]:
dt2017.groupby('school')['weight'].agg(sum)

In [ ]:
dt2018.groupby('school')['weight'].agg(sum)

In [ ]:
dt2017.groupby('education_attainment')['weight'].agg(sum)

In [ ]:
dt2018.groupby('education_attainment')['weight'].agg(sum)

In [ ]:
dt2017.groupby('citizenship')['weight'].agg(sum)

In [ ]:
dt2018.groupby('citizenship')['weight'].agg(sum)

In [ ]:
dt2017.groupby('disability_status')['weight'].agg(sum)

In [ ]:
dt2018.groupby('disability_status')['weight'].agg(sum)

In [ ]:
dt2017.groupby('disability_type')['weight'].agg(sum)

In [ ]:
dt2018.groupby('disability_type')['weight'].agg(sum)

In [55]:
dt2017.groupby('english_language')['weight'].agg(sum)

english_language
English Speaker                3467.0
Poor or No English language    7147.0
Name: weight, dtype: float64

In [ ]:
dt2018.groupby('english_language')['weight'].agg(sum)

In [58]:
dt2017.groupby('last_employed')['weight'].agg(sum)

last_employed
employed in the last year        3990.0
not employed in the last year    6624.0
Name: weight, dtype: float64

In [57]:
dt2018.groupby('last_employed')['weight'].agg(sum)

last_employed
employed in the last year        4102.0
not employed in the last year    5704.0
Name: weight, dtype: float64

In [ ]:
dt2017.groupby('parent_employment')['weight'].agg(sum)

In [ ]:
dt2018.groupby('parent_employment')['weight'].agg(sum)

In [ ]:
dt2017.groupby('marital_status')['weight'].agg(sum)

In [ ]:
dt2018.groupby('marital_status')['weight'].agg(sum)

In [ ]:
dt2017.groupby('child_born_las_year')['weight'].agg(sum)

In [ ]:
dt2018.groupby('child_born_las_year')['weight'].agg(sum)

In [ ]:
dt2017.groupby('presence_of_children')['weight'].agg(sum)

In [ ]:
dt2018.groupby('presence_of_children')['weight'].agg(sum)

In [56]:
dt2017.groupby('moved')['weight'].agg(sum)

moved
lived here last year    6725.0
moved here last year    3889.0
Name: weight, dtype: float64

In [ ]:
dt2018.groupby('moved')['weight'].agg(sum)

In [ ]:
dt2017.groupby('has_health_insurance')['weight'].agg(sum)

In [ ]:
dt2018.groupby('has_health_insurance')['weight'].agg(sum)

In [ ]:
dt2017.groupby('public_assistance_income_past_year')['weight'].agg(sum)

In [ ]:
dt2018.groupby('public_assistance_income_past_year')['weight'].agg(sum)

In [68]:
f17 = dt2017['gender']=='female'
f2017 = dt2017[f17]

f18 = dt2018['gender']=='female'
f2018 = dt2018[f18]

m17=dt2017['gender']=='male'
m2017=dt2017[m17]

m18=dt2018

,puma,rtrim,agep,gender,race_or_ethnicity,school,education_attainment,employment_status,disability_status,disability_type,...,english_language,last_employed,parent_employment,marital_status,child_born_las_year,presence_of_children,moved,has_health_insurance,public_assistance_income_past_year,weight
3,11614,King County (Southwest)--Auburn City & Lakeland,24.0,male,white,Has not attended in last 3 months,No HS or GED,Unemployed/not in labor force,no reported disability,None reported,...,Poor or No English language,employed in the last year,None,never married,None,no own children,moved here last year,yes,0.0,4.0
15,11614,King County (Southwest)--Auburn City & Lakeland,24.0,male,hispanic,Has not attended in last 3 months,post HS education,Unemployed/not in labor force,no reported disability,None reported,...,English Speaker,not employed in the last year,None,married,None,no own children,moved here last year,no,0.0,14.0
16,11614,King County (Southwest)--Auburn City & Lakeland,24.0,male,hispanic,Has not attended in last 3 months,No HS or GED,Unemployed/not in labor force,no reported disability,None reported,...,Poor or No English language,not employed in the last year,None,never married,None,no own children,lived here last year,no,0.0,5.0
21,11610,"King County (Central)--Renton City, Fairwood, ...",24.0,male,white,Has not attended in last 3 months,post HS education,Unemployed/not in labor force,no reported disability,None reported,...,Poor or No English language,not employed in the last year,None,never married,None,no own children,lived here last year,yes,0.0,22.0
25,11612,"King County (Far Southwest)--Federal Way, Des ...",24.0,male,white,Has not attended in last 3 months,HS diploma or GED,Unemployed/not in labor force,no reported disability,None reported,...,English Speaker,not employed in the last year,None,never married,None,no own children,moved here last year,no,0.0,37.0


In [64]:
f2017.groupby('agep')['weight'].agg(sum)

agep
16.0     153.0
17.0     150.0
18.0     646.0
19.0     675.0
20.0     496.0
21.0     595.0
22.0     823.0
23.0    1212.0
24.0     915.0
Name: weight, dtype: float64

In [71]:
pd.read_sql("""SELECT DISTINCT eng
            FROM pums_2017""",conn)

,eng
0,4
1,3
2,None
3,2
4,1
